In [12]:
import networkx as nx
import numpy as np
import pandas as pd

In [13]:
edges = pd.read_csv('HepPh/edges.csv')
nodes = pd.read_csv('HepPh/nodes.csv')

print(edges.shape, edges.columns)
print(nodes.shape)

(421578, 2) Index(['source', 'target'], dtype='object')
(34546, 4)


In [14]:
edge_list = edges[['source', 'target']].values.tolist()

G = nx.DiGraph()

G.add_edges_from(edge_list)

for _, row in nodes.iterrows():
    G.add_node(row['index'], name=row['name'], date=row['date'], pos=row['_pos'])


print('Number of edges: ', G.number_of_edges())
print('Number of nodes: ', G.number_of_nodes())

Number of edges:  421578
Number of nodes:  34546


In [15]:
import numpy as np

# Function for random selection based on in-degree
def ran_indeg_strategy(G, num_pivots):
    # Get the in-degree of each node
    indegrees = dict(G.in_degree())
    all_nodes = list(G.nodes)
    
    # Create a list of probabilities proportional to node in-degrees
    degree_sum = sum(indegrees.values())
    probabilities = [indegrees[node] / degree_sum for node in all_nodes]
    
    # Randomly select pivots with probability proportional to their in-degree
    pivots = np.random.choice(all_nodes, size=num_pivots, replace=False, p=probabilities)
    
    return pivots

In [16]:
# Function for random selection based on out-degree
def ran_outdeg_strategy(G, num_pivots):
    # Get the out-degree of each node
    outdegrees = dict(G.out_degree())
    all_nodes = list(G.nodes)
    
    # Create a list of probabilities proportional to node out-degrees
    degree_sum = sum(outdegrees.values())
    probabilities = [outdegrees[node] / degree_sum for node in all_nodes]
    
    # Randomly select pivots with probability proportional to their out-degree
    pivots = np.random.choice(all_nodes, size=num_pivots, replace=False, p=probabilities)
    
    return pivots

In [17]:
def pagerank_pivot_selection(G, num_pivots):
    """
    Select pivots based on PageRank scores.
    
    Parameters:
    - G: NetworkX directed graph (DiGraph)
    - num_pivots: Number of pivots to select
    
    Returns:
    - List of pivot nodes
    """
    # Calculate PageRank scores
    pagerank_scores = nx.pagerank(G)
    
    # Sort nodes by PageRank scores in descending order
    sorted_nodes = sorted(pagerank_scores, key=pagerank_scores.get, reverse=True)
    
    # Select the top nodes as pivots
    pivots = sorted_nodes[:num_pivots]
    return pivots

In [18]:
pivots = pagerank_pivot_selection(G, 3)
pivots

[3892, 2274, 9250]

In [19]:
def estimate_closeness_centrality(G, pivots):
    """
    Estimate closeness centrality using selected pivots.

    Parameters:
    - G: NetworkX graph
    - pivots: List of pivot nodes

    Returns:
    - Dictionary of estimated closeness centrality for each node
    """
    closeness_estimates = {}
    for node in G.nodes:
        distances = []
        for pivot in pivots:
            try:
                # Compute shortest path length
                distance = nx.shortest_path_length(G, source=pivot, target=node)
            except nx.NetworkXNoPath:
                # Assign a large value if no path exists
                distance = float('inf')
            distances.append(distance)
        
        # Compute average distance, ignoring infinite distances
        finite_distances = [d for d in distances if d != float('inf')]
        if finite_distances:
            avg_distance = sum(finite_distances) / len(finite_distances)
            closeness_estimates[node] = 1 / avg_distance if avg_distance > 0 else 0
        else:
            closeness_estimates[node] = 0  # Node is unreachable from all pivots
    return closeness_estimates

In [20]:
# Euclidean distance between exact and estimated centrality
def euclidean_distance(exact, estimated):
    return np.linalg.norm(exact-estimated)

In [ ]:
exact_closeness = nx.closeness_centrality(G)

In [21]:
def run_experiment(G, pivot_strategy, runs = 20):
    print('Calculating Closeness centrality')
    print('Closeness centrality: ', exact_closeness)
     # Calculate the total number of nodes
    num_nodes = G.number_of_nodes()
    num_pivots_list = [int(num_nodes / 20 * i) for i in range(1, 21)]
    results = []
    for i in range(runs):
        print('run ', i, '/', runs)
        for num_pivots in num_pivots_list:
            print('running with ', num_pivots, 'pivots')
            pivots = pivot_strategy(G, num_pivots)
            estimated_closeness = estimate_closeness_centrality(G, pivots)
            euclidean_dist = euclidean_distance(exact_closeness, estimated_closeness)
            results.append({
                "num_pivots": num_pivots,
                "euclidean_distance": euclidean_dist
            })

results = run_experiment(G, pagerank_pivot_selection)
results

Calculating Closeness centrality
Closeness centrality:  {0: 0.06527714140987805, 1: 0.10476018150956502, 2: 0.07715415478160952, 3: 0.07430068449023527, 4: 0.07806948021927218, 5: 0.0625198475798347, 6: 0.061669302119917335, 7: 0.06987701824167906, 8: 0.0722155476581398, 9: 0.06089802341514276, 10: 0.06590581115763208, 11: 0.06587897250068554, 12: 0.12451302082358671, 16: 0.08257925111731268, 17: 0.07417305341709589, 18: 0.057747642741380924, 19: 0.09303192417088918, 28: 0.13576173127289398, 29: 0.13799743826781824, 30: 0.12115805133761363, 31: 0.14024079884107044, 32: 0.1280559650773432, 33: 0.12689958100389562, 34: 0.10990883046177922, 35: 0.11749003663383381, 36: 0.06340800772365351, 37: 0.08713897648914277, 20: 0.12437445439446806, 21: 0.1195574384985271, 22: 0.10393538104443874, 23: 0.12212723763308485, 24: 0.11419355397154957, 25: 0.1017545305041412, 26: 0.0653274763189455, 13: 0.12751594693760224, 14: 0.06768610576512732, 15: 0.05592330630355416, 27: 0.07132468949970393, 38: 0.0

TypeError: unsupported operand type(s) for -: 'dict' and 'dict'